In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
df = pd.read_csv('df.csv',index_col=0)
df.head(10)

,match_id,time,gold,top_gold,jgl_gold,mid_gold,adc_gold,sup_gold,kill_total,assist_total,opp_kill_total,opp_assist_total,r_inhib_count,r_baron_count,r_tower_count,r_herald_count,r_drag_count,b_inhib_count,b_baron_count,b_tower_count,b_herald_count,b_dragon_count,kda,opp_kda,result
0,001a1f289e3bab22_b,0.0,2415.0,475.0,475.0,475.0,475.0,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,001a1f289e3bab22_b,1.0,2430.0,475.0,475.0,475.0,475.0,530.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,001a1f289e3bab22_b,2.0,2751.0,532.0,552.0,532.0,532.0,603.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,001a1f289e3bab22_b,3.0,3929.0,849.0,817.0,791.0,686.0,786.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,001a1f289e3bab22_b,4.0,5219.0,1099.0,1099.0,1107.0,976.0,938.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
5,001a1f289e3bab22_b,5.0,6418.0,1466.0,1278.0,1352.0,1196.0,1126.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.00,1.5,0.0
6,001a1f289e3bab22_b,6.0,8156.0,1667.0,1708.0,1574.0,1687.0,1520.0,2.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,3.5,0.0
7,001a1f289e3bab22_b,7.0,9520.0,1970.0,2097.0,1708.0,1995.0,1750.0,2.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.00,3.5,0.0
8,001a1f289e3bab22_b,8.0,10681.0,2147.0,2414.0,1988.0,2244.0,1888.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.75,4.5,0.0
9,001a1f289e3bab22_b,9.0,12036.0,2605.0,2533.0,2321.0,2535.0,2042.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.75,4.5,0.0


In [3]:
df.shape

(286908, 25)

## Fit Scaler to Full Dataset


In [70]:
# this df contains 2 games worth of rows
dft = df.head(76).copy()

In [71]:
# this df contains 1 game worth of rows
dfs = df.head(38).copy()

In [72]:
from sklearn.preprocessing import StandardScaler

data = dft.iloc[:,2:-1]
scaler = StandardScaler()
scaler.fit(data)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [54]:
dft.head()

,match_id,time,gold,top_gold,jgl_gold,mid_gold,adc_gold,sup_gold,kill_total,assist_total,opp_kill_total,opp_assist_total,r_inhib_count,r_baron_count,r_tower_count,r_herald_count,r_drag_count,b_inhib_count,b_baron_count,b_tower_count,b_herald_count,b_dragon_count,kda,opp_kda,result
0,001a1f289e3bab22_b,0.0,2415.0,475.0,475.0,475.0,475.0,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,001a1f289e3bab22_b,1.0,2430.0,475.0,475.0,475.0,475.0,530.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,001a1f289e3bab22_b,2.0,2751.0,532.0,552.0,532.0,532.0,603.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,001a1f289e3bab22_b,3.0,3929.0,849.0,817.0,791.0,686.0,786.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,001a1f289e3bab22_b,4.0,5219.0,1099.0,1099.0,1107.0,976.0,938.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Split into Train/Test Sets

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.7, random_state=0)

In [ ]:
for train_index, test_index in sss.split(array_a, array_res):
    X_train, X_test = array_a[train_index], array_a[test_index]
    y_train, y_test = array_res[train_index], array_res[test_index]

In [55]:
cols = list(dft.columns.values)
hold_out_cols = ['match_id', 'time', 'result']
ho_df = dft[hold_out_cols].copy()

In [56]:
cols_scale = ['gold', 'top_gold', 'jgl_gold', 'mid_gold', 'adc_gold', 'sup_gold', 'kill_total',
 'assist_total', 'opp_kill_total', 'opp_assist_total', 'r_inhib_count', 'r_baron_count', 'r_tower_count',
 'r_herald_count', 'r_drag_count', 'b_inhib_count', 'b_baron_count', 'b_tower_count', 'b_herald_count',
 'b_dragon_count', 'kda', 'opp_kda']

In [58]:
data = scaler.transform(data)

In [59]:
sdf = pd.DataFrame(data)
sdf.columns = cols_scale
sdf.head()

,gold,top_gold,jgl_gold,mid_gold,adc_gold,sup_gold,kill_total,assist_total,opp_kill_total,opp_assist_total,r_inhib_count,r_baron_count,r_tower_count,r_herald_count,r_drag_count,b_inhib_count,b_baron_count,b_tower_count,b_herald_count,b_dragon_count,kda,opp_kda
0,-1.493275,-1.483882,-1.566537,-1.452469,-1.430093,-1.531184,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018
1,-1.492426,-1.483882,-1.566537,-1.452469,-1.430093,-1.525186,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018
2,-1.474246,-1.468547,-1.541935,-1.438914,-1.416533,-1.495995,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018
3,-1.407531,-1.383268,-1.457266,-1.377318,-1.379898,-1.422819,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018
4,-1.334473,-1.316012,-1.367165,-1.302167,-1.310909,-1.362039,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018


In [60]:
print(len(sdf))
print(len(ho_df))

76
76


In [61]:
sdf = pd.merge(sdf, ho_df, left_index=True, right_index=True)
sdf.head()

,gold,top_gold,jgl_gold,mid_gold,adc_gold,sup_gold,kill_total,assist_total,opp_kill_total,opp_assist_total,r_inhib_count,r_baron_count,r_tower_count,r_herald_count,r_drag_count,b_inhib_count,b_baron_count,b_tower_count,b_herald_count,b_dragon_count,kda,opp_kda,match_id,time,result
0,-1.493275,-1.483882,-1.566537,-1.452469,-1.430093,-1.531184,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018,001a1f289e3bab22_b,0.0,0.0
1,-1.492426,-1.483882,-1.566537,-1.452469,-1.430093,-1.525186,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018,001a1f289e3bab22_b,1.0,0.0
2,-1.474246,-1.468547,-1.541935,-1.438914,-1.416533,-1.495995,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018,001a1f289e3bab22_b,2.0,0.0
3,-1.407531,-1.383268,-1.457266,-1.377318,-1.379898,-1.422819,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018,001a1f289e3bab22_b,3.0,0.0
4,-1.334473,-1.316012,-1.367165,-1.302167,-1.310909,-1.362039,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018,001a1f289e3bab22_b,4.0,0.0


In [62]:
fcols = [ 'match_id', 'time','gold', 'top_gold', 'jgl_gold', 'mid_gold', 'adc_gold', 'sup_gold', 'kill_total',
 'assist_total', 'opp_kill_total', 'opp_assist_total', 'r_inhib_count', 'r_baron_count', 'r_tower_count',
 'r_herald_count', 'r_drag_count', 'b_inhib_count', 'b_baron_count', 'b_tower_count', 'b_herald_count',
 'b_dragon_count', 'kda', 'opp_kda', 'result']

In [64]:
sdf1 = sdf[fcols]

In [65]:
sdf.head()

,match_id,time,gold,top_gold,jgl_gold,mid_gold,adc_gold,sup_gold,kill_total,assist_total,opp_kill_total,opp_assist_total,r_inhib_count,r_baron_count,r_tower_count,r_herald_count,r_drag_count,b_inhib_count,b_baron_count,b_tower_count,b_herald_count,b_dragon_count,kda,opp_kda,result
0,001a1f289e3bab22_b,0.0,-1.493275,-1.483882,-1.566537,-1.452469,-1.430093,-1.531184,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018,0.0
1,001a1f289e3bab22_b,1.0,-1.492426,-1.483882,-1.566537,-1.452469,-1.430093,-1.525186,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018,0.0
2,001a1f289e3bab22_b,2.0,-1.474246,-1.468547,-1.541935,-1.438914,-1.416533,-1.495995,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018,0.0
3,001a1f289e3bab22_b,3.0,-1.407531,-1.383268,-1.457266,-1.377318,-1.379898,-1.422819,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018,0.0
4,001a1f289e3bab22_b,4.0,-1.334473,-1.316012,-1.367165,-1.302167,-1.310909,-1.362039,-1.290139,-1.079324,-1.290139,-1.079324,-0.57735,0.0,-1.212965,0.0,-1.527938,0.0,0.0,-1.163474,0.0,-3.139609,-1.284018,-1.284018,0.0


In [90]:
X = sdf.loc[:,sdf.columns != 'result'].values
y = sdf.loc[:,sdf.columns == 'result'].values

In [91]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.7, random_state=0)

for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [94]:
y_test

array([[ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

## Test process on smaller dataframes

In [66]:
def parse_game(grp):
    """Inputs are 1. a groupby group object
    Output is a list of 3d arrays, each element contains 10 rows (which are equivalent to minutes 
       for this dataset). The function tracks the length of each group and stops at length minus 1. 
       After a group is finished the function performs the same process on the next group.
    """  
    t = 10
    start = 0
   
    length = len(grp)

    while t < length:
        
        # take a 10 row chunk and convert it to array
        
        data = grp.iloc[:,2:][start:t].values
   
        df_list.append(data)
        result_list.append(grp.iloc[:,-1:][start:t].values)
        
        # stop when t reaches length - 1
        start += 1
        t += 1
    
    return df_list  

In [67]:
df_list = []
result_list = []

gb = sdf.groupby('match_id')

gb.apply(lambda group: parse_game(group))

match_id
001a1f289e3bab22_b    [[[-1.49327540239, -1.48388166173, -1.56653668...
001a1f289e3bab22_r    [[[-1.49327540239, -1.48388166173, -1.56653668...
dtype: object

In [68]:
length_r = len(result_list)
print(len(result_list))
length_res = len(result_list)
array_res = np.array(result_list)
array_res = array_res.reshape(length_r,10,1)

84


In [73]:
length = len(df_list)
print(length)
array_a = np.array(df_list)
array_a = array_a.reshape((length,10,23))

84


In [74]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.7, random_state=0)

In [75]:
for train_index, test_index in sss.split(array_a, array_res):
    X_train, X_test = array_a[train_index], array_a[test_index]
    y_train, y_test = array_res[train_index], array_res[test_index]

ValueError: Found array with dim 3. Estimator expected <= 2.